In [1]:
import pandas as pd
import numpy as np
import numpy_financial as npf
import random
import math
import arrow

In [25]:
from __future__ import annotations
from typing import Union, Optional

import math

import arrow
import numpy_financial as npf
import pandas as pd


class Loan:
    """Loan class to create instances of installment loans.
     To be initialized with the following parameters:
        1. loan_amount:float - The Original loan amount (principal) disbursed
        on the loan date.
        2. int_rate:float - original rate of interest applicable on the
        principal outstanding.
        3. fees:float - Origination fees charged at the time of booking,
        expressed as a % of original loan amount.
        4. term:float - The original term of the loan per schedule.
        5. segment:str - The approx risk category of the loan. Broadly mapped
        to six FICO_score groups. Configurable via config.yaml for categories'
        6. channel:str - Indicator variable to identify if the loan was booked
        through a free channel or a paid channel.
        7. loan_dt:str - Date of loan disbursement.
        8. freq:str - Frequency of repayment. Monthly, Quarterly etc. Valid
        values can be accessed via the class variable loan.valid_pmt_freq
        """

    valid_pmt_freq = {
        'W': 'Weekly payments', '2W': 'Fortnightly payments',
        'M': 'Monthly payments', 'BM': 'Bi-monthly payments',
        'Q': 'Quarterly payments', 'H': 'Semi-annual payments',
        'Y': 'Annual payments',
    }
    freq_offset = {
        'W': pd.DateOffset(days=7, months=0),
        '2W': pd.DateOffset(days=14, months=0),
        'M': pd.DateOffset(days=0, months=1),
        'BM': pd.DateOffset(days=0, months=2),
        'Q': 'Q', 'H': '2Q', 'Y': 'Y',
    }
    period_offset = {
        'W': 7.0/30, '2W': 14.0/30,
        'M': 1, 'BM': 2, 'Q': 3, 'H': 6, 'Y': 12,
    }

    def __init__(
        self, loan_amt: float, int_rate: float, term: float, loan_dt: str,
            freq: str = 'M', fees_pct: Optional[float] = 0.0,
            segment: Optional[str] = 'c', channel: Optional[str] = 'free',
    ):
        self.loan_amt = loan_amt
        self.int_rate = int_rate
        self.fees_pct = fees_pct
        self.term = term
        self.segment = segment
        self.channel = channel
        self.loan_dt = arrow.get(loan_dt).datetime
        self.freq = freq
        self._offset = self.freq_offset[self.freq]
        self._periods = math.ceil(self.term/self.period_offset[self.freq])
        self._period_int_rate = self.int_rate*self.period_offset[self.freq]/12
        self.pmt = -npf.pmt(
            self._period_int_rate,
            self._periods, self.loan_amt,
        )

    def get_cfsch(self) -> pd.DataFrame:
        """Method to get the original scheduled of cashflows for a given loan.
        For monthly frequency (most common), it assumes that the dues date are
        on the same day of the month every month.
        Usage: loan.get_schedule()"""
        df = pd.DataFrame()
        df['dates'] = pd.Series(
            pd.date_range(
                self.loan_dt, freq=self._offset, periods=self._periods+1,
            ),
        ).shift(-1).dropna()
        df['period'] = df.index+1
        df['interest_pmt'] = - \
            npf.ipmt(
                self._period_int_rate,
                df['period'], self._periods, self.loan_amt,
            )
        df['principal_pmt'] = - \
            npf.ppmt(
                self._period_int_rate,
                df['period'], self._periods, self.loan_amt,
            )
        df['closing_principal'] = self.loan_amt-df['principal_pmt'].cumsum()
        df['opening_principal'] = df['closing_principal'].shift(
            1,
        ).fillna(self.loan_amt)
        return df

    @property
    def wal(self) -> float:
        """Returns the weighted average life of the loan (in months) based on
        the original cashflow schedule.
        The [WAL](https://en.wikipedia.org/wiki/Weighted-average_life) of the
        loan can be defined as the average number of months it takes for the
        principal of the loan
        to be repaid, if the borrower repays by the original schedule."""
        _cfs = self.get_cfsch()
        return (_cfs['principal_pmt']*_cfs['period']).sum() * \
            self.period_offset[self.freq]/self.loan_amt

    @property
    def apr(self) -> float:
        """Returns the Annual percentage rate (APR) of the loan based on the
        original cashflow schedule.
        The [APR](https://en.wikipedia.org/wiki/Annual_percentage_rate) of the
        loan can be defined as the
        total financial cost of the loan (including fees) divided by the WAL of
        the loan."""
        return self.int_rate+(self.fees_pct/(self.wal/12))

In [26]:
ir, t, fr, fe = (0.0599, 36, 'M', 0.05)
l1 = Loan(loan_amt=10000, int_rate=ir, term=t, loan_dt='2022-12-12',
                      freq=fr, fees_pct=fe)

In [29]:
print(f'WAL is equal to {l1.wal}')
print(f'APR is equal to {l1.apr}')

# l1.pmt
# l1.int_rate
# l1.fees_pct

WAL is equal to 19.037056255381792
APR is equal to 0.09141747790997778


In [28]:
l1.get_cfsch()

,dates,period,interest_pmt,principal_pmt,closing_principal,opening_principal
0,2023-01-12 00:00:00+00:00,1,49.916667,254.257400,9.745743e+03,10000.000000
1,2023-02-12 00:00:00+00:00,2,48.647498,255.526568,9.490216e+03,9745.742600
2,2023-03-12 00:00:00+00:00,3,47.371995,256.802071,9.233414e+03,9490.216032
3,2023-04-12 00:00:00+00:00,4,46.090125,258.083942,8.975330e+03,9233.413961
4,2023-05-12 00:00:00+00:00,5,44.801856,259.372211,8.715958e+03,8975.330019
5,2023-06-12 00:00:00+00:00,6,43.507156,260.666910,8.455291e+03,8715.957808
6,2023-07-12 00:00:00+00:00,7,42.205994,261.968073,8.193323e+03,8455.290898
7,2023-08-12 00:00:00+00:00,8,40.898336,263.275730,7.930047e+03,8193.322825
8,2023-09-12 00:00:00+00:00,9,39.584152,264.589915,7.665457e+03,7930.047095
9,2023-10-12 00:00:00+00:00,10,38.263407,265.910659,7.399547e+03,7665.457181


In [ ]:
channel_prop = {'free':0.8, 'paid':0.2}
channel_cost = {'free':0.0, 'paid':0.05}
term_prop = {'36':0.7, '60':0.3}
term_prem = {'36':0.0, '60':0.01} #reference term premium from treasury site
fee = {'No Fees': 0, '5% fees': 0.05, '10% fees': 0.1}

segment_ScoreMapping = {'f':'FICO:300-550', 'e':'FICO:550-650', 'd':'FICO:650-699',
                'c':'FICO:700-749', 'b':'FICO:750-799', 'a':'FICO:800-850'}

segment_prop = {'f':{'300-550':8.4%}, 'e':{'550-650':16.3%}, 'd':{'650-699':12.5%},
                'c':{'700-749':16.4%}, 'b':{'750-799':23.1%}, 'a':{'800-850':23.3%}} #six segments tied to chargeoffs and prepayment rates
chOff_mean = {'a':1%, 'b':1%, 'c':4.4%, 'd':8.9%, 'e':mean(22.5%, 15.8%), 'f':mean(28.4%, 41%)} # researchgate reference from 2000-2002
#establish a relationship between chargeoff and prepayment rates or FICO and prepayment rates
prepay_mean = {'a':45%, 'b':40%, 'c':35%, 'd':30%, 'e':25%, 'f':20%} # purely arbitrary


In [9]:
#Generating the prepayment curve and the Loss curve

# Losses peak by 9 months an36 std of about 3 months for a 36 month loan = 9/36 = 0.25; var = 9 mthsq = 9/36^2 = 0.00695 
# prepayments peak by 12 months and std by 6 months for a 36 month loan = 12/36 = 0.33; var = 9 mthsq = 36/36^2 = 0.0278

# alpha = mean*((mean*(1-mean)/var)-1)
# beta = (1-mean)*((mean*(1-mean)/var)-1)

# alpha_co = 0.25*((0.25*(1-0.25)/0.00695)-1)
# beta_co = (1-0.25)*((0.25*(1-0.25)/0.00695)-1)

# alpha_ppay = 0.33*((0.33*(1-0.33)/0.0278)-1)
# beta_ppay = (1-0.33)*((0.33*(1-0.33)/0.0278)-1)


In [22]:
# #sampling from a beta distribution
# x = math.floor(36*random.betavariate(alpha_co, beta_co))
# x

10

In [36]:
# y = math.floor(36*random.betavariate(alpha_ppay, beta_ppay))
# y

7